In [57]:
##### import numpy as np
from IPython.display import display, HTML

CSS = """
.output {
    align-items: center;
    div.output_area {
    width: 30%;
}
}
"""

HTML('<style>{}</style>'.format(CSS))

import pandas as pd
%matplotlib inline
from matplotlib import pyplot
from matplotlib.pyplot import plot
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


data_=pd.read_csv('comparative_emotion_analyzer.csv')
#data_
data=data_.drop(['Unnamed: 0','Song_ID','Artists','Artists_Id','Lengthofsing_ms','Modemajor1_happier','Release_date','Album_type','source','Popularity'],axis=1)
data['Mode']=data_['Modemajor1_happier']
for i in range(1,data.shape[1]):
    data.iloc[:,i]=data.iloc[:,i].apply(lambda x:round(x,3))
#data

n = int(input('Enter how many most frequent songs users listened in past month! '))
m = int(input('How many steps closer to your mood? '))
z = int(input('How many closest recommendations, you think we should offer?'))


# def song_recommendation(data,n):
#     data=data.iloc[0:n,:].reset_index().drop('index',axis=1)
    
def two_least_similar_choice(data):
    #print(data)
    cosim=cosine_similarity(data.iloc[:,1:],data.iloc[:,1:])
    #j=1
    res2=list()
    min_=cosim.min()
    #print(len(cosim))
    for i in range(len(cosim)):
        #print('i',i)
        for k in range(i,len(cosim)):
            #print('j',j)
            if cosim[i][k]==min_:
                res2.extend([i,k])
                #print(res2)
                #print(res)
        #print(i)

        #j+=1  
        #print(j)
    #print(i)
    #print(k)
    a=list(set(res2))
    #print(a)
    #print(data)
    song1=data['Song'][a[0]]
    song2=data['Song'][a[1]]
    


    #Have create buttons here and take the input
    print(f"Chose among songs:   {song1}   &   {song2} .") 
    selection=input('Which one? ')
    return selection

# def most_similar_songset(data,song):
#     n=data.shape[0]
    
def similarity(data):
    return cosine_similarity(data.iloc[:,1:],data.iloc[:,1:])

def recursive(data,z,m):
    count=m
    #print('recursive entry',data.shape)
    if count<1:
        print('You most likely feel like listening:')
        print(data['Song'])
        return None
        
    k=input(f'{data.shape[0]} songs? : Y or N')
    if k=='Y' or k=='y':
        print('You most likely feel like listening:')
        print(data['Song'])
        return None
    if data.shape[0]<z:
        print('You most likely feel like listening:')
        print (data['Song'])
        return None
    else:
        if count==2:
            data=data.iloc[:,:7]
            
        if count==1:
            data=pd.concat([pd.DataFrame(data['Song']),pd.DataFrame(data['energy']),data.loc[:,'valence':'tempo']],axis=1)

        cosim = similarity(data)
        sel=two_least_similar_choice(data)
        ind=data.set_index('Song').index.get_loc(sel)
        #print(ind)
        a=list(cosim[ind])
        if len(a)>0:
            list1=sorted(range(len(a)), key=lambda i: a[i], reverse=True)[0:len(a)//2]#BECAUSE FIRST IS THE SONG ITSELF WITH COSIN_SIM==1
        #list1
        else:
            print('You most likely feel like listening:')
            print(data['Song'])
            return None
        final_list=list()
        for i in list1:
            final_list.append(data.iloc[i,:].tolist())
        data_final=pd.DataFrame(final_list,columns=data.columns)
        count-=1
        recursive(data_final,z,count)
        

def final_model(data,n,m,z):
    model_data=data.iloc[0:25*n,:]
    model_data_norm= StandardScaler().fit_transform(model_data.iloc[:,1:])
    df1=model_data['Song']
    df1.reset_index(drop=True,inplace=True)
    df2=pd.DataFrame(model_data_norm[:,:],columns=list(model_data.columns)[1:])
    df2.reset_index(drop=True,inplace=True)
    data_final=pd.concat([df1,df2],axis=1)
    
    for i in range(1,data_final.shape[1]):
        data_final.iloc[:,i]=data_final.iloc[:,i].apply(lambda x:round(x,3))
    data_final=data_final.iloc[0:n,:].reset_index().drop('index',axis=1)
    
    #stripping the spaces in song names
    data_final['Song']=data_final['Song'].apply(lambda x: x.strip())
    k=z
    print(data_final.shape)
    recursive(data_final,k,m)
    #Choosing top n songs
final_model(data,n,m,z)

Enter how many most frequent songs users listened in past month! 60
How many steps closer to your mood? 3
How many closest recommendations, you think we should offer?3
(60, 18)
60 songs? : Y or Nn
Chose among songs:   Circles   &   everything i wanted .
Which one? everything i wanted
30 songs? : Y or Nn
Chose among songs:   Goodbyes   &   Bad Bad Bad .
Which one? Goodbyes
15 songs? : Y or Nn
Chose among songs:   Bandit   &   Lover .
Which one? Bandit
You most likely feel like listening:
0               Bandit
1               Panini
2               Ransom
3             Goodbyes
4         Did It Again
5    Sanguine Paradise
6           Trampoline
Name: Song, dtype: object
